In [ ]:
import numpy as np
import cv2
from base64 import b64decode
import io
from PIL import Image as PILImage
from IPython.display import display, Javascript

def capture_photo(quality=0.8):
    js = Javascript('''
        async function capturePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({ video: true });

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();


            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);


            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            const context = canvas.getContext('2d');

            while (true) {
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                // Check if 'q' key is pressed
                if (await new Promise((resolve) => document.onkeypress = resolve) == 'q') {
                    break;
                }
            }

            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')

    display(js)
    data = eval_js('capturePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    img = PILImage.open(io.BytesIO(binary))
    return np.array(img)

In [ ]:
def camera_calibration(images, chessboard_size, square_size):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    obj_points = []  # 3D 객체 포인트
    img_points = []  # 2D 이미지 포인트

    objp = np.zeros((1, chessboard_size[0] * chessboard_size[1], 3), np.float32)
    objp[0, :, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

    for image in images:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

        if ret:
            corners2 = cv2.cornerSubPix(
                gray, corners, (11, 11), (-1, -1), criteria
            )
            # 코너 좌표 계산
            img_points.append(corners2)
            obj_points.append(objp)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        obj_points, img_points, gray.shape[::-1], None, None
    )

    return mtx, dist

In [ ]:
def calculate_intrinsic_parameters(mtx, dist, image_size):
    new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, image_size, 1, image_size)

    return new_camera_mtx